# Confirmed Cases and Deaths Code
### Latest update of script on 2023-02-10

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np
from datetime import datetime
from get_dict_function import get_dict 

#pd options
pd.set_option("mode.chained_assignment", "raise")
pd.set_option('display.max_colwidth', 100)
pd.set_option("display.max_rows", 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## FIPS code

In [2]:
# importing fips data. and rearranging the columns
fips = pd.read_csv('../FIPS/state_county_fips.csv')

# inspect the object
fips.head()

fips.shape
# remove below for previous county-level FIPS code file but tentative keep the code
'''
fips = fips.iloc[:, [1, 3, 2, 4, 5]]

# Convert the 'integers' to integers in fips file.
for i in range(fips.shape[0]):
    if fips.iloc[i, 4] != 'unknown':
        fips.iloc[i, 4] = int(fips.iloc[i, 4])
        fips.iloc[i, 2] = int(fips.iloc[i, 2])

fips = fips.drop(fips[fips.fips == 0].index)
fips = fips.drop(fips[fips.fips == "unknown"].index)
fips = fips.reset_index()
fips = fips.iloc[0:3142, 1:]
'''

,stfips,ctyfips,stname,ctyname,fips
0,1,1,Alabama,Autauga County,1001
1,1,3,Alabama,Baldwin County,1003
2,1,5,Alabama,Barbour County,1005
3,1,7,Alabama,Bibb County,1007
4,1,9,Alabama,Blount County,1009


(3220, 5)

'\nfips = fips.iloc[:, [1, 3, 2, 4, 5]]\n\n# Convert the \'integers\' to integers in fips file.\nfor i in range(fips.shape[0]):\n    if fips.iloc[i, 4] != \'unknown\':\n        fips.iloc[i, 4] = int(fips.iloc[i, 4])\n        fips.iloc[i, 2] = int(fips.iloc[i, 2])\n\nfips = fips.drop(fips[fips.fips == 0].index)\nfips = fips.drop(fips[fips.fips == "unknown"].index)\nfips = fips.reset_index()\nfips = fips.iloc[0:3142, 1:]\n'

# Part I: County-Level Data and Dictionary

## Step 1: Update county-level data from CSSE

In [3]:
cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
cases.drop(cases.columns[[0,1,2, 3, 5, 6, 7, 8, 9, 10]], axis=1, inplace=True)

cases.rename(columns={"FIPS":"fips"}, inplace=True)

for col in cases.columns[1:]:
    cases.rename(columns={col: "confirmed_cases_" + datetime.strptime(col, "%m/%d/%y").strftime('%Y%m%d')}, inplace=True)

cases = pd.merge(fips, cases, on="fips", how="left")
#cases

In [4]:
deaths = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
deaths.drop(deaths.columns[[0,1,2, 3, 5, 6, 7, 8, 9, 10, 11]], axis=1, inplace=True)

deaths.rename(columns={"FIPS":"fips"}, inplace=True)

for col in deaths.columns[1:]:
    deaths.rename(columns={col: "deaths_" + datetime.strptime(col, "%m/%d/%y").strftime('%Y%m%d')}, inplace=True)

deaths = pd.merge(fips, deaths, on="fips", how="left")
#deaths

In [5]:
deaths = deaths.iloc[:, 5:]
csse = pd.concat([cases, deaths], axis=1)

In [6]:
# rename 'fips' column to 'scfips' to clarify it's state-county FIPS
csse.rename(columns = {'fips': 'scfips'}, inplace = True)
csse

,stfips,ctyfips,stname,ctyname,scfips,confirmed_cases_20200122,confirmed_cases_20200123,confirmed_cases_20200124,confirmed_cases_20200125,confirmed_cases_20200126,...,deaths_20230131,deaths_20230201,deaths_20230202,deaths_20230203,deaths_20230204,deaths_20230205,deaths_20230206,deaths_20230207,deaths_20230208,deaths_20230209
0,1,1,Alabama,Autauga County,1001,0,0,0,0,0,...,230,230,230,230,230,230,230,230,230,230
1,1,3,Alabama,Baldwin County,1003,0,0,0,0,0,...,723,723,723,723,723,723,723,723,723,723
2,1,5,Alabama,Barbour County,1005,0,0,0,0,0,...,103,103,103,103,103,103,103,103,103,103
3,1,7,Alabama,Bibb County,1007,0,0,0,0,0,...,109,109,109,109,109,109,109,109,109,109
4,1,9,Alabama,Blount County,1009,0,0,0,0,0,...,261,261,261,261,261,261,261,261,261,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72,145,Puerto Rico,Vega Baja Municipio,72145,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3216,72,147,Puerto Rico,Vieques Municipio,72147,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3217,72,149,Puerto Rico,Villalba Municipio,72149,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3218,72,151,Puerto Rico,Yabucoa Municipio,72151,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 2: Update CDC county-level cumulative provisional deaths data

In [7]:
cdcDeaths = pd.read_csv('https://data.cdc.gov/api/views/kn79-hsxy/rows.csv?accessType=DOWNLOAD')

cdcDeaths

,Date as of,Start Date,End Date,State,County name,FIPS County Code,Urban Rural Code,Deaths involving COVID-19,Deaths from All Causes,Footnote
0,02/08/2023,01/01/2020,02/04/2023,AK,Aleutians East Borough,2013,Noncore,NaN,15.00,One or more data cells have counts between 1-9 and have been suppressed in accordance with NCHS ...
1,02/08/2023,01/01/2020,02/04/2023,AK,Anchorage Municipality,2020,Medium metro,600.00,5357.00,NaN
2,02/08/2023,01/01/2020,02/04/2023,AK,Bethel Census Area,2050,Noncore,32.00,255.00,NaN
3,02/08/2023,01/01/2020,02/04/2023,AK,Denali Borough,2068,Noncore,NaN,14.00,One or more data cells have counts between 1-9 and have been suppressed in accordance with NCHS ...
4,02/08/2023,01/01/2020,02/04/2023,AK,Dillingham Census Area,2070,Noncore,NaN,73.00,One or more data cells have counts between 1-9 and have been suppressed in accordance with NCHS ...
...,...,...,...,...,...,...,...,...,...,...
3080,02/08/2023,01/01/2020,02/04/2023,WY,Sweetwater County,56037,Micropolitan,111.00,1059.00,NaN
3081,02/08/2023,01/01/2020,02/04/2023,WY,Teton County,56039,Micropolitan,19.00,301.00,NaN
3082,02/08/2023,01/01/2020,02/04/2023,WY,Uinta County,56041,Micropolitan,23.00,469.00,NaN
3083,02/08/2023,01/01/2020,02/04/2023,WY,Washakie County,56043,Noncore,31.00,254.00,NaN


In [8]:
# rename variables
cdcDeaths.rename(columns = {'State': 'stabbr', 'County name': 'ctyname', 'FIPS County Code': 'scfips',\
                            'Urban Rural Code': 'urbanrural', 'Deaths involving COVID-19': 'death_covid',\
                            'Deaths from All Causes': 'death_all', 'Start Date': 'death_startdate',\
                            'End Date': 'death_enddate'}, \
                 inplace = True)

In [9]:
# format dates
cdcDeaths['death_startdate'] = pd.to_datetime(cdcDeaths['death_startdate'])
cdcDeaths['death_startdate'] = cdcDeaths['death_startdate'].dt.strftime('%Y%m%d')
cdcDeaths['death_enddate'] = pd.to_datetime(cdcDeaths['death_enddate'])
cdcDeaths['death_enddate'] = cdcDeaths['death_enddate'].dt.strftime('%Y%m%d')

In [10]:
# extract end date
endDate = list(set(cdcDeaths['death_enddate']))[0]

# add end date to variable names
cdcDeaths.rename(columns = {'death_covid': 'death_covid_' + endDate, 'death_all': 'death_all_' + endDate}, \
                 inplace = True)

In [11]:
# keep selected variables only
cdcDeaths = cdcDeaths[['stabbr', 'scfips', 'urbanrural', 'death_startdate', 'death_covid_' + endDate, \
                       'death_all_' + endDate]].copy()

In [12]:
cdcDeaths

,stabbr,scfips,urbanrural,death_startdate,death_covid_20230204,death_all_20230204
0,AK,2013,Noncore,20200101,NaN,15.00
1,AK,2020,Medium metro,20200101,600.00,5357.00
2,AK,2050,Noncore,20200101,32.00,255.00
3,AK,2068,Noncore,20200101,NaN,14.00
4,AK,2070,Noncore,20200101,NaN,73.00
...,...,...,...,...,...,...
3080,WY,56037,Micropolitan,20200101,111.00,1059.00
3081,WY,56039,Micropolitan,20200101,19.00,301.00
3082,WY,56041,Micropolitan,20200101,23.00,469.00
3083,WY,56043,Noncore,20200101,31.00,254.00


## Step 3: Merge county-level data

In [13]:
# specify fips to be scfips
fips.rename(columns = {'fips': 'scfips'}, inplace = True)

In [14]:
# merge fips with csse
csse = fips.merge(csse, on = ['stfips', 'stname', 'ctyfips', 'ctyname', 'scfips'], how = 'outer')

In [15]:
# merge with cdc deaths
final = csse.merge(cdcDeaths, on = 'scfips', how = 'left')

In [16]:
final.columns

Index(['stfips', 'ctyfips', 'stname', 'ctyname', 'scfips',
       'confirmed_cases_20200122', 'confirmed_cases_20200123',
       'confirmed_cases_20200124', 'confirmed_cases_20200125',
       'confirmed_cases_20200126',
       ...
       'deaths_20230205', 'deaths_20230206', 'deaths_20230207',
       'deaths_20230208', 'deaths_20230209', 'stabbr', 'urbanrural',
       'death_startdate', 'death_covid_20230204', 'death_all_20230204'],
      dtype='object', length=2240)

In [17]:
final = final[['stfips', 'stabbr', 'stname', 'ctyfips', 'ctyname', 'scfips', 'urbanrural'] \
              + [x for x in list(final.columns)[5:] if x not in ['stabbr', 'urbanrural']]]

In [18]:
final['stfips'] = final['stfips'].astype('int')
final['scfips'] = final['scfips'].astype('int')
final

,stfips,stabbr,stname,ctyfips,ctyname,scfips,urbanrural,confirmed_cases_20200122,confirmed_cases_20200123,confirmed_cases_20200124,...,deaths_20230203,deaths_20230204,deaths_20230205,deaths_20230206,deaths_20230207,deaths_20230208,deaths_20230209,death_startdate,death_covid_20230204,death_all_20230204
0,1,AL,Alabama,1,Autauga County,1001,Medium metro,0,0,0,...,230,230,230,230,230,230,230,20200101,189.00,1644.00
1,1,AL,Alabama,3,Baldwin County,1003,Small metro,0,0,0,...,723,723,723,723,723,723,723,20200101,636.00,7564.00
2,1,AL,Alabama,5,Barbour County,1005,Noncore,0,0,0,...,103,103,103,103,103,103,103,20200101,79.00,871.00
3,1,AL,Alabama,7,Bibb County,1007,Large fringe metro,0,0,0,...,109,109,109,109,109,109,109,20200101,46.00,600.00
4,1,AL,Alabama,9,Blount County,1009,Large fringe metro,0,0,0,...,261,261,261,261,261,261,261,20200101,102.00,1634.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,72,NaN,Puerto Rico,145,Vega Baja Municipio,72145,NaN,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3216,72,NaN,Puerto Rico,147,Vieques Municipio,72147,NaN,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3217,72,NaN,Puerto Rico,149,Villalba Municipio,72149,NaN,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
3218,72,NaN,Puerto Rico,151,Yabucoa Municipio,72151,NaN,0,0,0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [19]:
# output
final.to_csv("../Pandemic/county_casesAndDeaths.csv", index = False)

## Step 4: Update county-level dictionary

In [20]:
cDict = pd.read_csv('../Pandemic/county_data_dictionary.csv')

In [21]:
tDict = cDict[cDict['field'] == 'cases and deaths']

In [22]:
tDict.variable_name

0                       stfips
1                       stabbr
2                       stname
3                      ctyfips
4                      ctyname
5                       scfips
6                   urbanrural
7     confirmed_cases_yyyymmdd
8              deaths_yyyymmdd
9              death_startdate
10          death_all_20230128
11        death_covid_20230128
Name: variable_name, dtype: object

In [23]:
'''
# move urbanrural variable to after scfips on 4/11/2022: run once
tDict1 = tDict.iloc[:6, :]
tDict2 = tDict.iloc[9, :]
tDict3 = tDict.iloc[6:9, :]
tDict4 = tDict.iloc[10:, :]
tDict = tDict1.append(tDict2, ignore_index = True)
tDict = tDict.append(tDict3, ignore_index = True)
tDict = tDict.append(tDict4, ignore_index = True)
tDict
'''

'\n# move urbanrural variable to after scfips on 4/11/2022: run once\ntDict1 = tDict.iloc[:6, :]\ntDict2 = tDict.iloc[9, :]\ntDict3 = tDict.iloc[6:9, :]\ntDict4 = tDict.iloc[10:, :]\ntDict = tDict1.append(tDict2, ignore_index = True)\ntDict = tDict.append(tDict3, ignore_index = True)\ntDict = tDict.append(tDict4, ignore_index = True)\ntDict\n'

In [24]:
tDict.loc[10, 'variable_name'] = 'death_all_' + endDate
tDict.loc[11, 'variable_name'] = 'death_covid_' + endDate

tDict.variable_name

0                       stfips
1                       stabbr
2                       stname
3                      ctyfips
4                      ctyname
5                       scfips
6                   urbanrural
7     confirmed_cases_yyyymmdd
8              deaths_yyyymmdd
9              death_startdate
10          death_all_20230204
11        death_covid_20230204
Name: variable_name, dtype: object

In [25]:
tDict = get_dict(final, tDict)

0
Single column for this item...
1
Single column for this item...
2
Single column for this item...
3
Single column for this item...
4
Single column for this item...
5
Single column for this item...
6
Single column for this item...
7
Multiple columns for this item...
8
Multiple columns for this item...
9
Single column for this item...
10
Single column for this item...
11
Single column for this item...


In [26]:
tDict

,variable_name,label,file,field,source,link,start_column,end_column,start_date,end_date,filename_in_release4
0,stfips,State name,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,1,1,-1,-1,casesAndDeaths.csv
1,stabbr,State abbreviation,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,2,2,-1,-1,casesAndDeaths.csv
2,stname,State FIPS code,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,3,3,-1,-1,casesAndDeaths.csv
3,ctyfips,County name,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,4,4,-1,-1,casesAndDeaths.csv
4,ctyname,County FIPS code,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,5,5,-1,-1,casesAndDeaths.csv
5,scfips,FIPS code of state-county,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,6,6,-1,-1,casesAndDeaths.csv
6,urbanrural,NCHS urban-rural classification,county_casesAndDeaths.csv,cases and deaths,CDC,https://data.cdc.gov/NCHS/Provisional-COVID-19-Death-Counts-in-the-United-St/kn79-hsxy,7,7,-1,-1,NaN
7,confirmed_cases_yyyymmdd,No of confirmed cases on yyyymmdd,county_casesAndDeaths.csv,cases and deaths,CSSE,https://github.com/CSSEGISandData/COVID-19,8,1122,20200122,20230209,casesAndDeaths.csv
8,deaths_yyyymmdd,No of deaths on yyyymmdd,county_casesAndDeaths.csv,cases and deaths,CSSE,https://github.com/CSSEGISandData/COVID-19,1123,2237,20200122,20230209,casesAndDeaths.csv
9,death_startdate,First date of data period,county_casesAndDeaths.csv,cases and deaths,CDC,https://data.cdc.gov/NCHS/Provisional-COVID-19-Death-Counts-in-the-United-St/kn79-hsxy,2238,2238,-1,-1,NaN


In [27]:
cDict[cDict['field'] == 'cases and deaths'] = tDict.copy()

cDict

,variable_name,label,file,field,source,link,start_column,end_column,start_date,end_date,filename_in_release4
0,stfips,State name,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,1,1,-1,-1,casesAndDeaths.csv
1,stabbr,State abbreviation,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,2,2,-1,-1,casesAndDeaths.csv
2,stname,State FIPS code,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,3,3,-1,-1,casesAndDeaths.csv
3,ctyfips,County name,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,4,4,-1,-1,casesAndDeaths.csv
4,ctyname,County FIPS code,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,5,5,-1,-1,casesAndDeaths.csv
5,scfips,FIPS code of state-county,county_casesAndDeaths.csv,cases and deaths,Census,https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html,6,6,-1,-1,casesAndDeaths.csv
6,urbanrural,NCHS urban-rural classification,county_casesAndDeaths.csv,cases and deaths,CDC,https://data.cdc.gov/NCHS/Provisional-COVID-19-Death-Counts-in-the-United-St/kn79-hsxy,7,7,-1,-1,NaN
7,confirmed_cases_yyyymmdd,No of confirmed cases on yyyymmdd,county_casesAndDeaths.csv,cases and deaths,CSSE,https://github.com/CSSEGISandData/COVID-19,8,1122,20200122,20230209,casesAndDeaths.csv
8,deaths_yyyymmdd,No of deaths on yyyymmdd,county_casesAndDeaths.csv,cases and deaths,CSSE,https://github.com/CSSEGISandData/COVID-19,1123,2237,20200122,20230209,casesAndDeaths.csv
9,death_startdate,First date of data period,county_casesAndDeaths.csv,cases and deaths,CDC,https://data.cdc.gov/NCHS/Provisional-COVID-19-Death-Counts-in-the-United-St/kn79-hsxy,2238,2238,-1,-1,NaN


In [28]:
# output
cDict.to_csv("../Pandemic/county_data_dictionary.csv", index = False)

# Part II: State-Level Data and Dictionary

## Step 1: Update CDC provisional deaths data by place of death and state

In [29]:
deaths = pd.read_csv('https://data.cdc.gov/api/views/uggs-hy5q/rows.csv?accessType=DOWNLOAD')
deaths.columns

deaths['Footnote'].unique()

footnote = list(set(deaths.Footnote))[1]
footnote

Index(['Data As Of', 'Start Date', 'End Date', 'Group', 'Year', 'Month',
       'State', 'Place of Death', 'COVID-19 Deaths', 'Total Deaths',
       'Pneumonia Deaths', 'Pneumonia and COVID-19 Deaths', 'Influenza Deaths',
       'Pneumonia, Influenza, or COVID-19 Deaths', 'Footnote'],
      dtype='object')

array([nan,
       'One or more data cells have counts between 1-9 and have been suppressed in accordance with NCHS confidentiality standards.'],
      dtype=object)

'One or more data cells have counts between 1-9 and have been suppressed in accordance with NCHS confidentiality standards.'

In [30]:
# selected by month, state-level, and all-aces-of-deaths rows
deaths = deaths[deaths['Group'] == 'By Month'].copy()
deaths = deaths[deaths['Place of Death'] == 'Total - All Places of Death'].copy()
deaths = deaths[deaths['State'] != 'United States'].copy()
deaths.reset_index(drop = True, inplace = True)

In [31]:
# select columns
deaths = deaths[['State', 'Year', 'Month', 'COVID-19 Deaths', 'Total Deaths',\
                 'Pneumonia Deaths', 'Pneumonia and COVID-19 Deaths', 'Influenza Deaths',\
                 'Pneumonia, Influenza, or COVID-19 Deaths']].copy()

In [32]:
# generate year-month variable
deaths['Year'] = deaths['Year'].astype('Int64')
deaths['Month'] = deaths['Month'].astype('Int64')
# turn 1- or 2-digit month in integer into 2-digit mongh string
deaths['Month'] = [str(x).zfill(2) for x in list(deaths.Month)]
# concatenate year and month info
deaths['year_month'] = deaths['Year'].astype('str') + deaths['Month']
deaths.drop(columns = ['Year', 'Month'], inplace = True)

In [33]:
# rename variables before reshape
deaths.rename(columns = {'Total Deaths': 'death_all_',\
                         'COVID-19 Deaths': 'death_covid_',\
                         'Pneumonia Deaths': 'death_pneu_',\
                         'Pneumonia and COVID-19 Deaths': 'death_cp_',\
                         'Influenza Deaths': 'death_flu_',\
                         'Pneumonia, Influenza, or COVID-19 Deaths': 'death_cpf_'},\
              inplace = True)

In [34]:
# get summary statistics before reshape to see if any suspicious records
smry = deaths[['death_all_', 'death_covid_', 'death_pneu_', 'death_cp_',\
               'death_flu_', 'death_cpf_']].describe().round(decimals = 2)
# check if any null values in all-cause death counts
print(smry['death_all_'].isnull().values.any()) # false, good to proceed
# inspect summary statistics manually
#smry.to_csv('~/Documents/ra/HPC/HPC_datahub/release6/cdc_provisional_deaths_state_summary.csv', index = True)
smry

False


,death_all_,death_covid_,death_pneu_,death_cp_,death_flu_,death_cpf_
count,2009.00,1936.00,1993.00,1827.00,1072.00,1995.00
mean,5205.39,573.79,532.47,304.97,16.28,818.19
std,5468.79,1170.79,830.63,672.12,40.35,1329.12
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,1405.00,68.75,118.00,35.00,0.00,170.00
50%,3861.00,213.50,305.00,100.00,0.00,432.00
75%,6517.00,580.00,588.00,292.00,18.00,902.00
max,48157.00,20065.00,13803.00,12191.00,746.00,21688.00


In [35]:
# get unique items as a prepare for dictionary
itms = list(deaths.columns)
itms.remove('State')
itms.remove('year_month')
itms = ['cdc_' + x + 'yyyymm' for x in itms]

In [36]:
# reshape from long form to wide form
deaths = deaths.pivot(index = 'State', columns = 'year_month')
deaths.columns = [x[0] + x[1] for x in list(deaths.columns)]
deaths.reset_index(drop = False, inplace = True)
deaths.rename(columns = {'State': 'stname'}, inplace = True)

In [37]:
# merge with fips code
fips = pd.read_csv('../FIPS/state_territory_fips.csv')
tdf = fips.merge(deaths, on = 'stname', how = 'outer')
tdf.iloc[:, :3]
# combine NYC into NY state
tdf.loc[57, 'stname'] = 'New York'
tdf = tdf.groupby(by = ['stfips', 'stabbr', 'stname'], as_index = False).sum()

,stfips,stabbr,stname
0,1.00,AL,Alabama
1,2.00,AK,Alaska
2,4.00,AZ,Arizona
3,5.00,AR,Arkansas
4,6.00,CA,California
5,8.00,CO,Colorado
6,9.00,CT,Connecticut
7,10.00,DE,Delaware
8,11.00,DC,District of Columbia
9,12.00,FL,Florida


In [38]:
# add 'cdc' prefix to column names
tdf.columns = list(tdf.columns)[:3] + ['cdc_' + x for x in list(tdf.columns)[3:]]

In [39]:
tdf

,stfips,stabbr,stname,cdc_death_covid_202001,cdc_death_covid_202002,cdc_death_covid_202003,cdc_death_covid_202004,cdc_death_covid_202005,cdc_death_covid_202006,cdc_death_covid_202007,...,cdc_death_cpf_202205,cdc_death_cpf_202206,cdc_death_cpf_202207,cdc_death_cpf_202208,cdc_death_cpf_202209,cdc_death_cpf_202210,cdc_death_cpf_202211,cdc_death_cpf_202212,cdc_death_cpf_202301,cdc_death_cpf_202302
0,1.00,AL,Alabama,0.00,0.00,48.00,339.00,430.00,399.00,879.00,...,270.00,299.00,428.00,459.00,394.00,336.00,336.00,465.00,412.00,0.00
1,2.00,AK,Alaska,0.00,0.00,0.00,0.00,0.00,0.00,12.00,...,40.00,36.00,36.00,41.00,34.00,26.00,24.00,29.00,18.00,0.00
2,4.00,AZ,Arizona,0.00,0.00,64.00,405.00,507.00,899.00,2102.00,...,440.00,484.00,479.00,448.00,396.00,378.00,573.00,910.00,494.00,19.00
3,5.00,AR,Arkansas,0.00,0.00,10.00,61.00,78.00,169.00,274.00,...,200.00,210.00,287.00,315.00,284.00,263.00,289.00,408.00,324.00,0.00
4,6.00,CA,California,0.00,0.00,279.00,2021.00,2238.00,2065.00,3890.00,...,1901.00,2143.00,2767.00,2596.00,2138.00,1843.00,2417.00,3802.00,2246.00,0.00
5,8.00,CO,Colorado,0.00,0.00,107.00,880.00,465.00,184.00,152.00,...,334.00,360.00,339.00,298.00,307.00,300.00,444.00,486.00,334.00,0.00
6,9.00,CT,Connecticut,0.00,0.00,126.00,2457.00,1535.00,285.00,77.00,...,298.00,252.00,236.00,256.00,279.00,323.00,309.00,481.00,339.00,0.00
7,10.00,DE,Delaware,0.00,0.00,0.00,189.00,276.00,73.00,35.00,...,72.00,75.00,86.00,81.00,75.00,70.00,83.00,111.00,80.00,0.00
8,11.00,DC,District of Columbia,0.00,0.00,15.00,262.00,272.00,116.00,43.00,...,38.00,40.00,44.00,44.00,47.00,41.00,57.00,63.00,39.00,0.00
9,12.00,FL,Florida,0.00,0.00,191.00,1311.00,1131.00,1083.00,4980.00,...,1576.00,2062.00,2469.00,2442.00,1867.00,1711.00,1558.00,2013.00,1993.00,42.00


## Step 2: Update state-level data file

In [40]:
# read existing data
slData = pd.read_csv('../Pandemic/state_level_data.csv')

# read state-level dictionary
slDict = pd.read_csv('../Pandemic/state_data_dictionary.csv')

In [41]:
# find indices of state-level cases & deaths variables
rows = slDict.index[slDict['link'] == 'https://data.cdc.gov/NCHS/Provisional-COVID-19-Deaths-by-Place-of-Death-and-/uggs-hy5q'].tolist()
# get the range of indices
cols = []
for i in range(len(rows)):
    cols = cols + list(range(slDict.loc[rows[i],'start_column'] - 1,\
                             slDict.loc[rows[i], 'end_column']))
# double-check if consecutive
print(cols[0])
print(cols[-1])
cols == list(range(cols[0], cols[-1] + 1))
# if true, good to proceed: insert this deaths data to assigned location

# CAUTION: since the column number starts from 1, which is different from python's starting column index as 0,
#          need to minus one for the starting column to get the real column index. The end column index is good
#          as python doesn't include the higher bound anyway.

108
329


True

In [42]:
# split state-level data into two trunks: 
# (1) before the start of previous provisional death data
# (2) after the end of cases & deaths data
df0 = slData.iloc[:, :(cols[0])]
df2 = slData.iloc[:, (cols[-1] + 1):]

In [43]:
# columns before target for replacement (double checke to ensure no wrong indexing)
df0.columns

Index(['stname', 'stabbr', 'stfips', 'apm_cases_total_2018',
       'apm_cases_hispanic_2018', 'apm_cases_nhwhite_2018',
       'apm_cases_nhblack_2018', 'apm_cases_nhasian_2018',
       'apm_cases_totalDeathBy_20200609', 'apm_cases_totalDeathBy_20200623',
       ...
       'apm_cases_asianDeath_20201027', 'apm_cases_asianDeath_20201110',
       'apm_cases_asianDeath_20201124', 'apm_cases_asianDeath_20201208',
       'apm_cases_asianDeath_20201222', 'apm_cases_asianDeath_20210105',
       'apm_cases_asianDeath_20210119', 'apm_cases_asianDeath_20210202',
       'apm_cases_asianDeath_20210216', 'apm_cases_asianDeath_20210302'],
      dtype='object', length=108)

In [44]:
# target for replacement (double checke to ensure no wrong indexing)
slData.iloc[:, cols[0]:(cols[-1] + 1)].columns

Index(['cdc_death_covid_202001', 'cdc_death_covid_202002',
       'cdc_death_covid_202003', 'cdc_death_covid_202004',
       'cdc_death_covid_202005', 'cdc_death_covid_202006',
       'cdc_death_covid_202007', 'cdc_death_covid_202008',
       'cdc_death_covid_202009', 'cdc_death_covid_202010',
       ...
       'cdc_death_cpf_202204', 'cdc_death_cpf_202205', 'cdc_death_cpf_202206',
       'cdc_death_cpf_202207', 'cdc_death_cpf_202208', 'cdc_death_cpf_202209',
       'cdc_death_cpf_202210', 'cdc_death_cpf_202211', 'cdc_death_cpf_202212',
       'cdc_death_cpf_202301'],
      dtype='object', length=222)

In [45]:
# columns after target for replacement (double checke to ensure no wrong indexing)
df2.columns

Index(['medical_covidtracking_positive_20200113',
       'medical_covidtracking_positive_20200114',
       'medical_covidtracking_positive_20200115',
       'medical_covidtracking_positive_20200116',
       'medical_covidtracking_positive_20200117',
       'medical_covidtracking_positive_20200118',
       'medical_covidtracking_positive_20200119',
       'medical_covidtracking_positive_20200120',
       'medical_covidtracking_positive_20200121',
       'medical_covidtracking_positive_20200122',
       ...
       'apm_vac_latino_20210503', 'apm_vac_latino_20210517',
       'apm_vac_indig_20210405', 'apm_vac_indig_20210419',
       'apm_vac_indig_20210503', 'apm_vac_indig_20210517',
       'apm_vac_other_20210405', 'apm_vac_other_20210419',
       'apm_vac_other_20210503', 'apm_vac_other_20210517'],
      dtype='object', length=11750)

In [46]:
# merge provisional deaths data with the first half
df1 = df0.merge(tdf, on = ['stfips', 'stabbr', 'stname'], how = 'left')
# concatenate df0 and df1 on columns
fnl = pd.concat([df1, df2], axis = 1)

In [47]:
# output updated data
fnl.to_csv('../Pandemic/state_level_data.csv', index = False)

## Step 3: Update state-level dictionary

In [48]:
# update dictionary
fdct = get_dict(fnl, slDict)

0
Single column for this item...
1
Single column for this item...
2
Single column for this item...
3
Single column for this item...
4
Single column for this item...
5
Single column for this item...
6
Single column for this item...
7
Single column for this item...
8
Multiple columns for this item...
9
Multiple columns for this item...
10
Multiple columns for this item...
11
Multiple columns for this item...
12
Multiple columns for this item...
13
Multiple columns for this item...
14
Multiple columns for this item...
15
Multiple columns for this item...
16
Multiple columns for this item...
17
Multiple columns for this item...
18
Multiple columns for this item...
19
Multiple columns for this item...
20
Multiple columns for this item...
21
Multiple columns for this item...
22
Multiple columns for this item...
23
Multiple columns for this item...
24
Multiple columns for this item...
25
Multiple columns for this item...
26
Multiple columns for this item...
27
Multiple columns for this item..

In [49]:
fdct

,variable_name,label,field,source,link,start_column,end_column,start_date,end_date,filename_in_release4,footnote
0,stname,State name,state identifier,Census,https://www2.census.gov/geo/docs/reference/state.txt,1,1,-1,-1,NaN,NaN
1,stabbr,State abbreviation,state identifier,Census,https://www2.census.gov/geo/docs/reference/state.txt,2,2,-1,-1,NaN,NaN
2,stfips,State FIPS code,state identifier,Census,https://www2.census.gov/geo/docs/reference/state.txt,3,3,-1,-1,NaN,NaN
3,apm_cases_total_2018,Total population in 2018,cases and deaths,APM,https://www.apmresearchlab.org/covid/deaths-by-race,4,4,-1,-1,ApmColorOfCoronavirus.csv,NaN
4,apm_cases_hispanic_2018,Hispanic pooulation in 2018,cases and deaths,APM,https://www.apmresearchlab.org/covid/deaths-by-race,5,5,-1,-1,ApmColorOfCoronavirus.csv,NaN
...,...,...,...,...,...,...,...,...,...,...,...
135,apm_vac_white_yyyymmdd,Number of White persons vaccinated,vaccination,APM,https://www.apmresearchlab.org/covid/vaccines-by-race,12067,12070,20210405,20210517,NaN,NaN
136,apm_vac_asian_yyyymmdd,Number of Asian persons vaccinated,vaccination,APM,https://www.apmresearchlab.org/covid/vaccines-by-race,12071,12074,20210405,20210517,NaN,NaN
137,apm_vac_latino_yyyymmdd,Number of Latino persons vaccinated,vaccination,APM,https://www.apmresearchlab.org/covid/vaccines-by-race,12075,12078,20210405,20210517,NaN,NaN
138,apm_vac_indig_yyyymmdd,Number of Indigenous persons vaccinated,vaccination,APM,https://www.apmresearchlab.org/covid/vaccines-by-race,12079,12082,20210405,20210517,NaN,NaN


In [50]:
# output updated dictionary
fdct.to_csv('../Pandemic/state_data_dictionary.csv', index = False)